# The task is divided on 2 part:
- ## compare how parameters like: *country*, *professional*, *education* (formal and non-formal), *IT skill* and *work experience* impact on salary, 
- ## predict future salary for people whose salary value in `'NaN'`.

### Because schema has big number of different kind of questions (some of them are not connected with IT) not every columns are needed to make a job done.

In [1]:
import pandas as pd

In [2]:
# CSV file with people answers is too big to put into Github directly, so it will be insert as a ZIP file and uncompressed later in Python code

answers_file = pd.read_csv('survey_results_public.zip', compression='zip', index_col = False)
questions_file = pd.read_csv('survey_results_schema.csv', index_col = False)

## General inspection of data in *'answers_file'*

In [3]:
# Remove default limits of Pandas .head() settings
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

In [4]:
answers_file.head()

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,WorkPayCare,KinshipDevelopers,ChallengeMyself,CompetePeers,ChangeWorld,JobSeekingStatus,HoursPerWeek,LastNewJob,AssessJobIndustry,AssessJobRole,AssessJobExp,AssessJobDept,AssessJobTech,AssessJobProjects,AssessJobCompensation,AssessJobOffice,AssessJobCommute,AssessJobRemote,AssessJobLeaders,AssessJobProfDevel,AssessJobDiversity,AssessJobProduct,AssessJobFinances,ImportantBenefits,ClickyKeys,JobProfile,ResumePrompted,LearnedHiring,ImportantHiringAlgorithms,ImportantHiringTechExp,ImportantHiringCommunication,ImportantHiringOpenSource,ImportantHiringPMExp,ImportantHiringCompanies,ImportantHiringTitles,ImportantHiringEducation,ImportantHiringRep,ImportantHiringGettingThingsDone,Currency,Overpaid,TabsSpaces,EducationImportant,EducationTypes,SelfTaughtTypes,TimeAfterBootcamp,CousinEducation,WorkStart,HaveWorkedLanguage,WantWorkLanguage,HaveWorkedFramework,WantWorkFramework,HaveWorkedDatabase,WantWorkDatabase,HaveWorkedPlatform,WantWorkPlatform,IDE,AuditoryEnvironment,Methodology,VersionControl,CheckInCode,ShipIt,OtherPeoplesCode,ProjectManagement,EnjoyDebugging,InTheZone,DifficultCommunication,CollaborateRemote,MetricAssess,EquipmentSatisfiedMonitors,EquipmentSatisfiedCPU,EquipmentSatisfiedRAM,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,Strongly agree,Strongly disagree,Agree,Agree,Disagree,Agree,"I'm not actively looking, but I am open to new opportunities",0.0,Not applicable/ never,Very important,Very important,Important,Very important,Very important,Very important,Important,Very important,Very important,Very important,Very important,Very important,Somewhat important,Not very important,Somewhat important,Stock options; Vacation/days off; Remote options,Yes,Other,NaN,NaN,Important,Important,Important,Somewhat important,Important,Not very important,Not very important,Not at all important,Somewhat important,Very important,NaN,NaN,Tabs,NaN,Online course; Open source contributions,NaN,NaN,NaN,6:00 AM,Swift,Swift,NaN,NaN,NaN,NaN,iOS,iOS,Atom; Xcode,Turn on some music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somewhat satisfied,Not very satisfied,Not at all satisfied,Very satisfied,Satisfied,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stack Overflow,9.0,Deskto

## Getting informations about DataFrame (inproper or missing values and types of data)

In [5]:
answers_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Columns: 154 entries, Respondent to ExpectedSalary
dtypes: float64(6), int64(1), object(147)
memory usage: 60.4+ MB


---

## **Feature selection**
**Removing columns with data can be done in 2 ways:**    

*   by human intuition (read FULL description to questions and think about sense of using some part of them in work),
*   by specialized algorythms (it is fully automated without our control).

First option is much slower, but I think is more effective using this data set - if the final effect will not be satisfaction enough, we will give a chance for the algorythms.

### Looking for columns in *'answer_file'* DataFrame where percent number of missing values not exceed 25%

In [6]:
columns_length, columns_number = answers_file.shape[0], answers_file.shape[1]

def Nan_values_calculate(nan_percent):
    cols_list = []
    for col_name in (answers_file.columns):
        if ((answers_file[col_name].isnull().sum() / columns_length) <= nan_percent):
            cols_list.append(col_name)
    ## How many columns satisfy requirement
    print(f'{len(cols_list)} columns from {columns_number} of total have got equal or less than {int(100*nan_percent)}% missing values')
    return cols_list


## Result as a list with the names of useful columns
columns_list = Nan_values_calculate(0.25)
print('\n List:',columns_list, '\n')

18 columns from 154 of total have got equal or less than 25% missing values

 List: ['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University', 'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote', 'CompanySize', 'CompanyType', 'YearsProgram', 'YearsCodedJob', 'CareerSatisfaction', 'JobSatisfaction', 'PronounceGIF', 'ClickyKeys', 'TabsSpaces'] 



### Limiting number of columns to 18 is great, but this is only beginning before we can make build regressor model. 
### Next step is manually checking which of these columns are probably not important and maybe we need to add some extra columns (not founded by our previous filter function, but also worth our attention).

In [7]:
import numpy as np

In [8]:
## From columns label we take some suspicious names and read their description from 'questions_file'

columns_to_check = ['PronounceGIF', 'ClickyKeys', 'TabsSpaces', 'DeveloperType',
                'HaveWorkedLanguage']

for i in columns_to_check:
    idx = questions_file.index[questions_file['Column'] == i][0]
    print(f"Question from '{i}': {questions_file.loc[int(idx),'Question']}", '\n')

Question from 'PronounceGIF': How do you pronounce "GIF"? 

Question from 'ClickyKeys': If two developers are sharing an office, is it OK for one of them to get a mechanical keyboard with loud "clicky" keys? 

Question from 'TabsSpaces': Tabs or spaces? 

Question from 'DeveloperType': Which of the following best describe you? 

Question from 'HaveWorkedLanguage': Which of the following languages have you done extensive development work in over the past year, and which do you want to work in over the next year? 



### From original DataFrame we will take 6 extra columns which were not included in the list above: 
####    - 'Gender'
####    - 'Race'
####    - 'DeveloperType',
####    - 'HaveWorkedLanguage'
####    - 'Currency'
####    - 'Salary' (target)
### After joining all these columns, we may also remove 'Respondent' Series, because values inside it are simple numerical order (like index in rows number).

### Make a new ***'df_answers'***  DataFrame

In [9]:
extra_cols = ['Gender', 'Race', 'DeveloperType', 'HaveWorkedLanguage', 'Currency', 'Salary']

df_answers = pd.concat([answers_file[columns_list], answers_file[extra_cols]], axis=1)
df_answers = df_answers.drop(columns=['Respondent', 'PronounceGIF', 'ClickyKeys', 'TabsSpaces'], axis = 1)

## Info about our new dataset
df_answers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51392 entries, 0 to 51391
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Professional        51392 non-null  object 
 1   ProgramHobby        51392 non-null  object 
 2   Country             51392 non-null  object 
 3   University          51392 non-null  object 
 4   EmploymentStatus    51392 non-null  object 
 5   FormalEducation     51392 non-null  object 
 6   MajorUndergrad      42841 non-null  object 
 7   HomeRemote          44008 non-null  object 
 8   CompanySize         38922 non-null  object 
 9   CompanyType         38823 non-null  object 
 10  YearsProgram        51145 non-null  object 
 11  YearsCodedJob       40890 non-null  object 
 12  CareerSatisfaction  42695 non-null  float64
 13  JobSatisfaction     40376 non-null  float64
 14  Gender              35047 non-null  object 
 15  Race                33033 non-null  object 
 16  Deve

In [10]:
df_answers.to_csv('survey_results_public_clean.csv', index = False)
df_answers = pd.read_csv('survey_results_public_clean.csv', index_col = False)

df_answers.head()

,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,CareerSatisfaction,JobSatisfaction,Gender,Race,DeveloperType,HaveWorkedLanguage,Currency,Salary
0,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,Male,White or of European descent,NaN,Swift,NaN,NaN
1,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning a bachelor's degree,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,"Privately-held limited company, not in startup mode",9 to 10 years,NaN,NaN,NaN,Male,White or of European descent,NaN,JavaScript; Python; Ruby; SQL,British pounds sterling (£),NaN
2,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day each week","10,000 or more employees",Publicly-traded corporation,20 or more years,20 or more years,8.0,9.0,Male,White or of European descent,Other,Java; PHP; Python,British pounds sterling (£),113750.0
3,Professional non-developer who sometimes writes code,"Yes, both",United States,No,Employed full-time,Doctoral degree,A non-computer-focused engineering discipline,"Less than half the time, but at least one day each week","10,000 or more employees",Non-profit/non-governmental organization or private school/university,14 to 15 years,9 to 10 years,6.0,3.0,Male,White or of European descent,NaN,Matlab; Python; R; SQL,NaN,NaN
4,Professional developer,"Yes, I program as a hobby",Switzerland,No,Employed full-time,Master's degree,Computer science or software engineering,Never,10 to 19 employees,"Privately-held limited company, not in startup mode",20 or more years,10 to 11 years,6.0,8.0,NaN,NaN,Mobile developer; Graphics programming; Desktop applications developer,NaN,NaN,NaN
